In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import networkx as nx
import osmnx as ox
import geopandas as gpd
import networkx as nx
#from cartopy.geodesic import Geodesic
from shapely.geometry.polygon import Point, Polygon
from shapely.geometry import shape
from shapely.ops import unary_union
from shapely.geometry.multipolygon import MultiPolygon
import nhstravel.loaders.lsoaloader as lsoaloader
import folium
import geopandas as gpd

ModuleNotFoundError: No module named 'nhstravel'

In [ ]:
#load in the post code to lsoa look up sheet
postcode_lookup = pd.read_csv('../data/PCD_OA21_LSOA21_MSOA21_LAD_NOV22_UK_LU 3.csv', encoding = "ISO-8859-1")
#postcode_lookup = pd.read_csv('/Users/paul.carroll/Github/pycom/nhs_time_of_travel/data/pcd_lsoa21cd_nov22_en.csv', encoding = "ISO-8859-1")
postcode_lookup.head()

In [ ]:
#load in the hospital data set to get the postcodes for lookup
#load in the hospital sites data to be used to score potential viable locations and filter for Cambridge
'''
hospitals = pd.read_csv('data/Hospital.csv')
surrey_hospitals = hospitals.loc[hospitals['County'] == 'Surrey']
surrey_hospitals
'''

In [ ]:
#define the target location post codes 
target_address1 = 'CB2 8AF' #4 Trumpington Road
target_address2 = 'CB2 0AY' #Nuffield Hospital (central cambridge location)
list_of_target_addresses = [target_address1, target_address2]



In [ ]:
target_coords = []
for loc in list_of_target_addresses:
    target_coords.append(ox.geocode(loc))


target_coords

In [ ]:
'''annotating this cell to save where this is now'''

In [ ]:
#get the lsoa 2021 code from the lookup file
lsoa_codes = []
lsoa_names = []

for postcode in list_of_target_addresses:
    lsoa_codes.append(postcode_lookup.loc[postcode_lookup['pcds'] == postcode]['lsoa21cd'].values[0])
    lsoa_names.append(postcode_lookup.loc[postcode_lookup['pcds'] == postcode]['ladnm'].values[0])

lsoa_names


In [ ]:
#load in lsoa data using the loaders function in nhs travel
lsoa_with_population_pd1 = lsoaloader.build_lsoa_data_frame_for_area_england(lsoa_names[0])
#lsoa_with_population_pd1 = lsoaloader.build_lsoa_data_frame_for_area_england(lsoa_codes[0])#

remapped_lsoa1 = lsoaloader.load_geo_json_shapefiles_for_lsoas(lsoa_with_population_pd1, lsoa_names[0])
#remapped_lsoa1 = lsoaloader.load_geo_json_shapefiles_for_lsoas(lsoa_with_population_pd1, lsoa_codes[0])

In [ ]:
remapped_lsoa1.type


In [ ]:
target_coords[1]

In [ ]:
target_coords1 = target_coords[0]

#gd = Geodesic()
#bounding_poly1 = Polygon(gd.circle(lon=target_coords1[1], lat=target_coords1[0], radius=500))




target_point1 = Point(target_coords1[1], target_coords1[0])

neighboring_polys1 = {'lsoa_codes':[], 'population':[], 'polygons':[]}
for lsoa in remapped_lsoa1['features']:
    lsoa_polygon = shape(lsoa['geometry'])
    if lsoa_polygon.contains(target_point1) or lsoa_polygon.distance(target_point1) < 500:
        neighboring_polys1['lsoa_codes'].append(lsoa['properties']['LSOA21CD'])
        neighboring_polys1['population'].append(lsoa['properties']['all ages'])
        neighboring_polys1['polygons'].append(lsoa_polygon)

    '''if lsoa_polygon.contains(target_point1) or bounding_poly1.intersects(lsoa_polygon):
        neighboring_polys1['lsoa_codes'].append(lsoa['properties']['LSOA21CD'])
        neighboring_polys1['population'].append(lsoa['properties']['all ages'])
        neighboring_polys1['polygons'].append(lsoa_polygon)'''

In [ ]:
len(neighboring_polys1['lsoa_codes'])


In [ ]:
len(remapped_lsoa1['features'])

In [ ]:
#load in lsoa data using the loaders function in nhs travel
lsoa_with_population_pd2 = lsoaloader.build_lsoa_data_frame_for_area_england(lsoa_names[1])
remapped_lsoa2 = lsoaloader.load_geo_json_shapefiles_for_lsoas(lsoa_with_population_pd2, lsoa_names[1])

In [ ]:
target_coords2 = target_coords[1]

#gd = Geodesic()
#bounding_poly2 = Polygon(gd.circle(lon=target_coords2[1], lat=target_coords2[0], radius=500))

target_point2 = Point(target_coords2[1], target_coords2[0])

neighboring_polys2 = {'lsoa_codes':[], 'population':[], 'polygons':[]}
for lsoa in remapped_lsoa2['features']:
    lsoa_polygon = shape(lsoa['geometry'])
    if lsoa_polygon.contains(target_point2) or lsoa_polygon.distance(target_point2) < 500:
        neighboring_polys2['lsoa_codes'].append(lsoa['properties']['LSOA21CD'])
        neighboring_polys2['population'].append(lsoa['properties']['all ages'])
        neighboring_polys2['polygons'].append(lsoa_polygon)

In [ ]:
neighboring_polys_merged = neighboring_polys1['polygons']  + neighboring_polys2['polygons']
neighboring_multipoly = MultiPolygon(neighboring_polys_merged)
hull = neighboring_multipoly.convex_hull
hull

In [ ]:
hull_neighboring_polys = {'lsoa_codes':[], 'population':[], 'polygons':[]}
for lsoa in remapped_lsoa1['features']:
    lsoa_polygon = shape(lsoa['geometry'])
    if hull.intersects(lsoa_polygon):
        hull_neighboring_polys['lsoa_codes'].append(lsoa['properties']['LSOA21CD'])
        hull_neighboring_polys['population'].append(lsoa['properties']['all ages'])
        hull_neighboring_polys['polygons'].append(lsoa_polygon)
hull_neighboring_polys

In [ ]:
len(hull_neighboring_polys)

In [ ]:
neighboring_polys_merged = hull_neighboring_polys['polygons']
bounding_poly_merged = unary_union(neighboring_polys_merged)

In [ ]:
bounding_poly_merged

In [ ]:
G = ox.graph_from_polygon(bounding_poly_merged)
#---------------------------------------------------------------------------------------
ox.plot_graph(G, figsize = (10,10));

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
nodes

In [ ]:
#convert the addresses to lat long coordinates and get the nearest node on the graph
target_coords1 = ox.geocode(target_address1)

target_node1 = ox.distance.nearest_nodes(G, target_coords1[1],target_coords1[0])

target_coords2 = ox.geocode(target_address2)
target_node2 = ox.distance.nearest_nodes(G, target_coords2[1],target_coords1[0])

list_of_target_nodes = [target_node1, target_node2]
list_of_target_coords = [target_coords1, target_coords2]

In [ ]:
nodes_sample1 = pd.DataFrame(columns = nodes.columns)
nodes_sample1.

In [ ]:
#get the dataframe of nodes that are in the neighboring polys and get the sample nodes with the corresponding population and lsoa codes
nodes_sample1 = pd.DataFrame(columns = nodes.columns)
list_of_lsoa_codes = []
list_of_pops = []

for j in range(nodes.shape[0]):

    for i in range(len(neighboring_polys1['polygons'])):
        lsoa = neighboring_polys1['polygons'][i]
    
        if lsoa.contains(nodes.iloc[j]['geometry']):
            nodes_sample1 = nodes_sample1.append(nodes.iloc[j])
            list_of_lsoa_codes.append(neighboring_polys1['lsoa_codes'][i])
            list_of_pops.append(neighboring_polys1['population'][i])
            continue

nodes_sample1['lsoa_codes'] = list_of_lsoa_codes
nodes_sample1['lsoa_population'] = list_of_pops

nodes_sample1 = nodes_sample1.drop(target_node1)

nodes_sample1

In [ ]:
#get the dataframe of nodes that are in the neighboring polys and get the sample nodes with the corresponding population and lsoa codes
nodes_sample1 = pd.DataFrame(columns = nodes.columns)
list_of_lsoa_codes = []
list_of_pops = []


for i in range(len(neighboring_polys1['polygons'])):
    lsoa = neighboring_polys1['polygons'][i]
    for j in range(nodes.shape[0]):
        if lsoa.contains(nodes.iloc[j]['geometry']):
            nodes_sample1 = nodes_sample1.append(nodes.iloc[j])
            list_of_lsoa_codes.append(neighboring_polys1['lsoa_codes'][i])
            list_of_pops.append(neighboring_polys1['population'][i])

nodes_sample1['lsoa_codes'] = list_of_lsoa_codes
nodes_sample1['lsoa_population'] = list_of_pops

nodes_sample1 = nodes_sample1.drop(target_node1)

nodes_sample1

In [ ]:
#get the dataframe of nodes that are in the neighboring polys and get the sample nodes with the corresponding population and lsoa codes
nodes_sample2 = pd.DataFrame(columns = nodes.columns)
list_of_lsoa_codes = []
list_of_pops = []


for i in range(len(neighboring_polys2['polygons'])):
    lsoa = neighboring_polys2['polygons'][i]
    for j in range(nodes.shape[0]):
        if lsoa.contains(nodes.iloc[j]['geometry']):
            nodes_sample2 = nodes_sample2.append(nodes.iloc[j])
            list_of_lsoa_codes.append(neighboring_polys2['lsoa_codes'][i])
            list_of_pops.append(neighboring_polys2['population'][i])

nodes_sample2['lsoa_codes'] = list_of_lsoa_codes
nodes_sample2['lsoa_population'] = list_of_pops

nodes_sample2 = nodes_sample2.drop(target_node2)

nodes_sample2

In [ ]:
nodes_sample = nodes_sample2
node = 554267
current_lsoa = nodes_sample['lsoa_codes'][node]
nodes_in_lsoa = nodes_sample.loc[nodes_sample['lsoa_codes']==current_lsoa].shape[0]
nodes_in_lsoa

In [ ]:
#get the dataframe of nodes that are in the neighboring polys and get the sample nodes with the corresponding population and lsoa codes
nodes_sample1 = pd.DataFrame(columns = nodes.columns)
nodes_sample2 = pd.DataFrame(columns = nodes.columns)

list_of_lsoa_codes1 = []
list_of_pops1 = []

list_of_lsoa_codes2 = []
list_of_pops2 = []

for lsoa in neighboring_polys_merged:
    for i in range(nodes.shape[0]):
        if lsoa.contains(nodes.iloc[i]['geometry']):
            if lsoa in neighboring_polys1['polygons']:
                index = neighboring_polys1['polygons'].index(lsoa)
                list_of_lsoa_codes1.append(neighboring_polys1['lsoa_codes'][index])
                list_of_pops1.append(neighboring_polys1['population'][index])
                nodes_sample1 = nodes_sample1.append(nodes.iloc[i])
            elif lsoa in neighboring_polys2['polygons']:
                index = neighboring_polys2['polygons'].index(lsoa)
                list_of_lsoa_codes2.append(neighboring_polys2['lsoa_codes'][index])
                list_of_pops2.append(neighboring_polys2['population'][index])
                nodes_sample2 = nodes_sample2.append(nodes.iloc[i])
                
            

nodes_sample1['lsoa_codes'] = list_of_lsoa_codes1
nodes_sample1['lsoa_population'] = list_of_pops1

#nodes_sample1 = nodes_sample1.drop(target_node1)


nodes_sample2['lsoa_codes'] = list_of_lsoa_codes2
nodes_sample2['lsoa_population'] = list_of_pops2

nodes_sample2 = nodes_sample2.drop(target_node2)



nodes_sample1


In [ ]:
#creating a function to calculate a score from a list of lengths calculated from the target node to each of the 100 sample nodes
def create_score(list_of_lengths, list_of_multipliers):
    score = 1000
    for l, m in zip(list_of_lengths, list_of_multipliers):
        deduction = (((l/1000)/4.5)*60) * m * 5#get the length in km divide by speed 4.5 km/h then divide by 60 to get time in minutes
        score = score - deduction #decrement the score by the derivation of time taken to each of the 100 nodes
        return score

In [ ]:
#define a function to calculate multiple shortest route lengths from the target node to each of the 100 sample nodes
def create_list_of_lengths(nodes_sample, target_node):
    list_of_lengths = []
    list_of_multipliers = []


    for node in nodes_sample.index:
        current_lsoa = nodes_sample['lsoa_codes'][node]
        nodes_in_lsoa = nodes_sample.loc[nodes_sample['lsoa_codes']==current_lsoa].shape[0]
        total_pop = nodes_sample['lsoa_population'].unique().sum()
        node_pop = nodes_sample['lsoa_population'][node]/nodes_in_lsoa
        multiplier = 1 - (node_pop/total_pop)
        try:
            length = nx.shortest_path_length(G, source=node, target=target_node, weight='length') #calculate route from target node to sample node
        except Exception as e:
            pass
        list_of_lengths.append(length) #append the length to the list
        list_of_multipliers.append(multiplier) #append the multipliers to the list for score creation
    
    return [list_of_lengths, list_of_multipliers]
    

In [ ]:
#run the function defined above to generate a list of lengths between the target nodes and sample nodes
#then generate the score for each and store them in a dictionary
target_scores = {}
target1_lengths = create_list_of_lengths(nodes_sample1, list_of_target_nodes[0])
target2_lengths = create_list_of_lengths(nodes_sample2, list_of_target_nodes[1])
target_scores['Site 1'] = create_score(target1_lengths[0], target1_lengths[1])
target_scores['Site 2'] = create_score(target2_lengths[0], target2_lengths[1])
target_scores

In [ ]:
#print the scores for each proposed target site
print('The score for Site 1: {} is {}'.format(target_address1, target_scores['Site 1']))
print('The score for Site 2: {} is {}'.format(target_address2, target_scores['Site 2']))

In [ ]:
#initiate dictionary to store routes for all 100 nodes for each proposed site
target_to_nodes_routes = {}
target_node_names = ['Site1', 'Site2']

In [ ]:
list_of_samples = [nodes_sample, nodes_sample]

In [ ]:
#generate multiple shortest routes for each site and store in dictionary for plotting
for site, target_node in zip(target_node_names, list_of_target_nodes):
    list_of_routes = []
    for node in nodes_sample.index:
        try:
            route = nx.shortest_path(G, source=node, target=target_node, weight='length') #calculate route from target node to sample node
        except Exception as el
        list_of_routes.append(route) #append the length to the list
    target_to_nodes_routes[site] = list_of_routes
    


In [ ]:
#plot the routes from Site 1 to all 100 sample nodes
fig, ax = ox.plot_graph_routes(G, target_to_nodes_routes['Site1'], route_linewidth=6, bgcolor='k')

In [ ]:
#plot the routes from Site 1 to all 100 sample nodes
fig, ax = ox.plot_graph_routes(G, target_to_nodes_routes['Site2'], route_linewidth=6, bgcolor='k')